In [1]:
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer


def get_actual_answer(input: str) -> str:
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))

    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(input)}],
        user_id=user.id,
    )

2025-05-18 16:13:54 - Loaded .env file
2025-05-18 16:13:57 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-18 16:13:57 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}


/home/ntp2003/.cache/pypoetry/virtualenvs/chatbot-tlcn-L8ehHy6a-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-18 16:13:58 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-18 16:13:59 - file_cache is only supported with oauth2client<4.0.0
2025-05-18 16:13:59 - file_cache is only supported with oauth2client<4.0.0


In [2]:
import os
import random
import json
from typing import Dict, List, Optional, Tuple, Union
import openai
from dotenv import load_dotenv
from service.wandb import client as wandb_client
from service.openai import  _client
# Load environment variables
load_dotenv()

# Setup OpenAI client
client = _client


class VietnameseUserSimulator:
    def __init__(self, persona_type: str = "random"):
        """
        Initialize the Vietnamese user simulator with a specific persona type.

        Args:
            persona_type: Type of persona to simulate ("detailed", "minimal", "impatient", "confused", or "random")
        """
        self.persona_type = (
            persona_type
            if persona_type != "random"
            else random.choice(["detailed", "minimal", "impatient", "confused"])
        )

        # Initialize persona traits using OpenAI API
        self._generate_persona()

        # Track conversation state
        self.conversation_history = []
        self.provided_contact_info = False
        self.product_interest_stated = False
        self.brand_preference_stated = False
        self.questions_stated = False
        self.ready_to_purchase = False

    def _generate_persona(self):
        """Generate a realistic Vietnamese customer persona using OpenAI API with structured output"""

        # Few-shot examples for different persona types
        persona_examples = {
            "detailed": {
                "name": "Nguyễn Minh Hải",
                "gender": "male",
                "email": "minhhai88@gmail.com",
                "phone": "0903456789",
                "age": 32,
                "occupation": "Software Engineer",
                "product_type": "laptop",
                "budget": 25000000,
                "preferred_brand": "Dell",
                "technical_knowledge": "high",
                "verbosity": "high",
                "patience": "high",
                "willingness_to_share_info": "high",
                "questions": [
                    "Sản phẩm có đi kèm bảo hành mở rộng không?",
                    "Có khuyến mãi nào áp dụng hiện tại không?",
                    "Tôi có thể chọn màu sắc không?",
                    "Máy có hỗ trợ nâng cấp phần cứng trong tương lai không?",
                    "Thời lượng pin thực tế khi sử dụng lập trình và chạy máy ảo là bao lâu?",
                    "Màn hình có hỗ trợ tần số quét cao không?",
                    "Có đi kèm hệ điều hành bản quyền không?",
                ],
                "communication_style": "Likes to discuss technical details, asks many questions about specifications, compares different models, provides complete information when asked",
            },
            "minimal": {
                "name": "Trần Thị Mai",
                "gender": "female",
                "email": "maimai@gmail.com",
                "phone": "0912345678",
                "age": 27,
                "occupation": "Office Worker",
                "product_type": "phone",
                "budget": 7000000,
                "preferred_brand": "Samsung",
                "technical_knowledge": "medium",
                "verbosity": "low",
                "patience": "medium",
                "willingness_to_share_info": "medium",
                "questions": [
                    "Giá đã bao gồm phụ kiện chưa?",
                    "Có được giảm giá nếu thanh toán qua ví điện tử không?",
                    "Máy có bao nhiêu màu để chọn?",
                    "Có được trả góp không?",
                ],
                "communication_style": "Provides short answers, often one word responses, focuses on price, hesitant to share personal information immediately",
            },
            "impatient": {
                "name": "Lê Văn Dũng",
                "gender": "male",
                "email": "levandung75@gmail.com",
                "phone": "0865432198",
                "age": 45,
                "occupation": "Business Owner",
                "product_type": "phone",
                "budget": 30000000,
                "preferred_brand": "iPhone",
                "technical_knowledge": "medium",
                "verbosity": "medium",
                "patience": "low",
                "willingness_to_share_info": "low",
                "questions": [
                    "Giá rẻ nhất hiện tại là bao nhiêu?",
                    "Có khuyến mãi gì không? Nói nhanh giúp tôi.",
                    "Tôi mua hôm nay có được tặng phụ kiện gì không?",
                    "Bảo hành chính hãng hay bên cửa hàng?",
                    "Máy có còn nguyên seal không?",
                ],
                "communication_style": "Asks about prices quickly, shows impatience with phrases like 'nhanh lên', focuses on getting the best deal, reluctant to share contact info until convinced of value",
            },
            "confused": {
                "name": "Phạm Thanh Hà",
                "gender": "female",
                "email": "hathanhpham@yahoo.com.vn",
                "phone": "0389765432",
                "age": 52,
                "occupation": "Teacher",
                "product_type": "laptop",
                "budget": 15000000,
                "preferred_brand": "HP",
                "technical_knowledge": "low",
                "verbosity": "high",
                "patience": "medium",
                "willingness_to_share_info": "medium",
                "questions": [
                    "Cái này dùng để soạn bài được không?",
                    "Nó có nhẹ không, tôi cần mang đi lại nhiều?",
                    "Bảo hành có khó làm thủ tục không?",
                    "Có cần phải mua thêm phần mềm gì nữa không?",
                    "Nếu tôi mua thì mấy ngày nhận được?",
                    "Có cài sẵn gì trong máy không?",
                ],
                "communication_style": "Often confuses technical terms, asks for clarification frequently, changes questionsring conversation, mixes up RAM and storage, repeats questions",
            },
        }
        # Create system prompt for persona generation
        system_prompt = (
            "You are an API that generates highly diverse and realistic Vietnamese customer personas for an e-commerce simulator. "
            "Each persona should be unique in background, age, gender, occupation, region, communication style, and buying behavior. "
            "Vary the level of technical knowledge, willingness to share information, and style of asking questions. "
            "Personas should include both urban and rural backgrounds, a range of ages (18-65), and different levels of digital literacy. "
            "Some personas may be very chatty, others very brief; some may be skeptical, others trusting; some may be price-focused, others care about brand or features. "
            "Generate a detailed persona for a Vietnamese customer interested in technology products (phone or laptop). "
            "Your output must be VALID JSON that matches the structure of the example. "
            "Use realistic Vietnamese names, contact information, and behavior patterns."
        )

        # Select a few-shot example based on persona type
        example = persona_examples[self.persona_type]
        example_json = json.dumps(example, ensure_ascii=False, indent=2)

        # Create user prompt with instructions and example
        user_prompt = (
            f"Generate a realistic and diverse Vietnamese customer persona with the following characteristics:\n"
            f"- Persona type: {self.persona_type}\n"
            f"- Product interest: Either phone or laptop\n"
            f"- Include realistic Vietnamese name, email, phone number\n"
            f"- Include appropriate budget in VND\n"
            f"- Include technical questions appropriate for the product\n"
            f"- Specify behavioral traits: technical_knowledge, verbosity, patience, willingness_to_share_info\n"
            f"- Vary region (urban/rural), occupation, and communication style for diversity\n"
            f"- Optionally, add unique quirks or preferences (e.g., prefers texting, dislikes calls, cares about eco-friendly packaging, etc.)\n"
            f"\nUse this structure as a guide (this just for reference, you can change the values and fields must be full filled):\n"
            f"{example_json}\n"
            f"\n'questions' in the JSON should be a list of questions the user might ask about the product when they choose one product from the suggested list.\n"
            f"'phone' and 'email' should be realistic Vietnamese formats.\n"
            f"\nRespond ONLY with a valid JSON object matching this structure but with different realistic values."
        )

        # Call OpenAI API for structured persona generation
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=1.0,
        )
        # Parse the response
        persona_data = json.loads(response.choices[0].message.content)
        
        print(f"Persona data: {json.dumps(persona_data, ensure_ascii=False, indent=2)}")
        # Set persona attributes
        self.name = persona_data.get("name", "Nguyễn Văn A")
        self.gender = persona_data.get("gender", "unknown")
        self.email = persona_data.get(
            "email", f"user{random.randint(100,999)}@gmail.com"
        )
        self.phone = persona_data.get(
            "phone", f"09{random.randint(10000000, 99999999)}"
        )
        self.age = persona_data.get("age", random.randint(18, 65))
        self.occupation = persona_data.get("occupation", "Unknown")
        self.product_type = persona_data.get(
            "product_type", random.choice(["phone", "laptop"])
        )
        self.budget = persona_data.get(
            "budget",
            (
                random.randint(2_000_000, 40_000_000)
                if self.product_type == "phone"
                else random.randint(10_000_000, 60_000_000)
            ),
        )
        self.preferred_brand = persona_data.get("preferred_brand", "")
        self.technical_knowledge = persona_data.get("technical_knowledge", "medium")
        self.verbosity = persona_data.get("verbosity", "medium")
        self.patience = persona_data.get("patience", "medium")
        self.willingness_to_share_info = persona_data.get(
            "willingness_to_share_info", "medium"
        )
        self.questions = persona_data.get("questions", {})
        self.communication_style = persona_data.get("communication_style", "")

    def _get_vietnamese_formality(self) -> Dict[str, str]:
        """Get Vietnamese formality based on persona"""
        # Choose pronouns based on persona
        # In Vietnamese, pronouns vary greatly and indicate social relationships
        options = [
            {"self": "Tôi", "you": "bạn"},  # Neutral/formal
            {"self": "Em", "you": "anh/chị"},  # Younger person to older
            {"self": "Mình", "you": "bạn"},  # Friendly/informal
        ]
        return random.choice(options)

    def generate_response(self, agent_message: str) -> str:
        """
        Generate a user response to the workflow agent's message.

        Args:
            agent_message: The message from the workflow agent

        Returns:
            The simulated user's response
        """
        self.conversation_history.append(
            {"role": "assistant", "content": agent_message}
        )

        # Use OpenAI to generate a user-like response, with explicit role instruction
        system_prompt = self._create_system_prompt()
        # Add explicit instruction to always answer as a customer, not as an agent
        system_prompt += (
            "\n\nIMPORTANT: You are the CUSTOMER in this conversation. "
            "Never answer as the agent or provide information as if you are the seller. "
            "Do NOT repeat, summarize, or rephrase the agent's information. "
            "Do NOT ask the agent questions on their behalf. "
            "Only ask questions, express needs, or respond as a buyer. "
            "If the agent asks if you want more info, either ask a new question, say 'ok', or indicate you want to buy, but never provide product details as if you are the seller or agent."
            "\nIf you are about to say something like 'Em đang quan tâm đến mẫu nào?' or any phrase that sounds like the agent, STOP and instead reply as a customer (e.g., ask about price, warranty, features, or say 'ok')."
            "\nNever use phrases like 'Em có thể giúp gì cho bạn', 'Bạn có muốn biết thêm thông tin gì không?', or any sentence that sounds like you are the agent."
        )

        messages = [
            {"role": "system", "content": system_prompt},
            *self.conversation_history,
        ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.9 if self.persona_type == "confused" else 0.7,
            max_tokens=150,
        )

        user_response = response.choices[0].message.content

        self.conversation_history.append({"role": "user", "content": user_response})
        self._update_conversation_state(user_response)
        return user_response

    def _get_few_shot_examples(self) -> List[Dict[str, str]]:
        """Get few-shot examples for the specific persona type"""
        examples = {
            "detailed": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Tôi đang cần mua một chiếc laptop mới để làm việc. Tôi làm trong lĩnh vực thiết kế đồ họa nên cần một chiếc máy có cấu hình mạnh.",
                },
                {
                    "role": "assistant",
                    "content": "Dạ vâng, tôi rất vui được tư vấn cho anh/chị. Đối với công việc thiết kế đồ họa, anh/chị quan tâm đến thương hiệu nào cụ thể không ạ?",
                },
                {
                    "role": "user",
                    "content": "Tôi thích các dòng máy của Dell hoặc ASUS. Tôi cần một máy có card đồ họa tốt, RAM ít nhất 16GB và bộ xử lý mạnh. Màn hình cũng cần có độ phân giải cao và độ phủ màu tốt. Budget của tôi khoảng 30 triệu.",
                },
            ],
            "minimal": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {"role": "user", "content": "Tìm điện thoại"},
                {
                    "role": "assistant",
                    "content": "Dạ vâng, bạn đang quan tâm đến điện thoại hãng nào ạ? Và bạn có yêu cầu gì về cấu hình và giá cả không ạ?",
                },
                {"role": "user", "content": "Samsung. Dưới 10tr"},
            ],
            "impatient": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Cần mua điện thoại gấp. Có iPhone 15 không?",
                },
                {
                    "role": "assistant",
                    "content": "Dạ chào anh/chị. Hiện tại cửa hàng có iPhone 15 nhiều phiên bản. Anh/chị quan tâm đến mẫu nào cụ thể ạ? iPhone 15, 15 Plus, 15 Pro hay 15 Pro Max?",
                },
                {
                    "role": "user",
                    "content": "Giá bao nhiêu? Nhanh lên, tôi không có nhiều thời gian. 15 Pro Max 256GB giá bao nhiêu?",
                },
            ],
            "confused": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Tôi muốn mua laptop nhưng không biết nên mua loại nào. Có nhiều RAM là tốt phải không?",
                },
                {
                    "role": "assistant",
                    "content": "Dạ vâng, RAM là một yếu tố quan trọng. Anh/chị dự định sử dụng laptop cho công việc gì ạ?",
                },
                {
                    "role": "user",
                    "content": "À tôi chỉ dùng để lướt web thôi. Nhưng đôi khi tôi cũng chỉnh sửa ảnh. RAM là bộ nhớ trong đúng không? Hay là ổ cứng nhỉ? Tôi cần ổ cứng lớn để lưu nhiều ảnh. Mà máy càng nhẹ càng tốt.",
                },
            ],
        }

        return examples.get(self.persona_type, examples["detailed"])


    def _create_system_prompt(self) -> str:
        """Create a system prompt for the API based on persona and conversation state"""
        formality = self._get_vietnamese_formality()

        prompt = f"""You are simulating a Vietnamese user who is interested in purchasing {"a phone" if self.product_type == "phone" else "a laptop"}.

PERSONA DETAILS:
- Type: {self.persona_type} 
- Name: {self.name}
- Product interest: {self.product_type}
- Preferred brand: {self.preferred_brand}
- Budget: {self.budget:,} VND (about {round(self.budget/23000, 2)} USD)
- Technical knowledge: {self.technical_knowledge}
- Patience level: {self.patience}
- Verbosity: {self.verbosity}
- Willingness to share contact info: {self.willingness_to_share_info}
- Vietnamese pronouns: refers to self as '{formality["self"]}', refers to agent as '{formality["you"]}'

Your responses should be primarily in Vietnamese with occasional English words (as is common in Vietnamese tech conversations). 
Use Vietnamese chat style and abbreviations when appropriate:
- "ko" for "không" (no)
- "ok" or "oki" for agreement
- "đc" for "được" (can/ok)
- "khoảng" abbreviated as "khoang" 
- Numbers + "tr" for millions (e.g., "15 tr" = 15 million VND)

PERSONA BEHAVIOR GUIDELINES:
"""

        # Add persona-specific behavior guidelines
        if self.persona_type == "detailed":
            prompt += """
- Provide detailed information about your needs and preferences
- Ask technical questions about the products
- Respond directly to questions and provide thorough information when asked
- Be polite and patient
- Willing to share contact information when it makes sense
"""
        elif self.persona_type == "minimal":
            prompt += """
- Keep responses very short, often just a few words
- Answer questions directly with minimal elaboration
- Sometimes use single word answers or short Vietnamese phrases
- Prefer simple language and avoid technical details
- Hesitant to share contact information immediately but will if pressed
"""
        elif self.persona_type == "impatient":
            prompt += """
- Show signs of impatience with phrases like "nhanh lên", "mau", etc.
- Ask about prices early and often
- Focus heavily on getting the best deal
- May sound abrupt or slightly demanding
- Reluctant to share contact information until convinced of good value
- Might try to negotiate or ask about discounts
"""
        elif self.persona_type == "confused":
            prompt += """
- Ask for clarification frequently
- Sometimes misunderstand basic product specifications
- Change your mind about questionsd-conversation
- Ask repeated questions about the same topics
- Get details wrong or confused (like mixing up RAM and storage)
- Somewhat hesitant to share contact information due to confusion
"""

        # Add information about product questions
        prompt += f"\nPRODUCT questions\nWhen shown specific products and you choose one product, ask relevant questions from this list based on what matters most to you:\n" 
        for i, value in enumerate(self.questions):
            prompt += f"- {i}: {value}\n"
            
        prompt += "\nExample:\n"
        prompt += "The suggested product is:\n1. A\n2. B\n3. C\n"
        prompt += "The user chose product 1. A\n"
        prompt += "The user might ask: one of the questions above\n"
        prompt += "\n\n"

        # Conversation state-specific guidance
        prompt += "\nCONVERSATION STATE GUIDELINES:\n"

        if not self.product_interest_stated:
            prompt += (
                "- When asked about product type, indicate your interest in "
                + self.product_type
                + "\n"
            )

        if self.product_interest_stated and not self.brand_preference_stated:
            prompt += f"- If asked about brand preference, mention {self.preferred_brand} or say you're open to suggestions\n"

        if not self.questions_stated:
            prompt += "- Be ready to mention your questions when the conversation moves to specific product features\n"

        if not self.provided_contact_info:
            if self.willingness_to_share_info == "high":
                prompt += "- Provide your contact info (email and phone) when asked or when it seems appropriate\n"
            elif self.willingness_to_share_info == "medium":
                prompt += "- Initially hesitate but eventually provide contact info when asked repeatedly\n"
            else:
                prompt += "- Be very reluctant to share contact info; may need significant convincing\n"

        prompt += """
IMPORTANT:
- Respond ONLY as the user would, do not include explanations or meta-commentary
- Your response should look like a genuine Vietnamese customer chat message
- Do not use quotation marks around your response
- Sometimes include typos as would be natural in chat
- If the agent asks for your contact information and you're ready to provide it, give your name, email, and phone number
"""

        return prompt

    def _update_conversation_state(self, response: str) -> None:
        """
        Update conversation state based on the user's response using OpenAI API for more accurate analysis.
        """
        # Compose system prompt for state extraction with detailed field descriptions
        system_prompt = (
            "You are an API that analyzes a Vietnamese user's chat message and extracts conversation state flags. "
            "Given the user's latest message and the persona details, output a JSON object with these boolean fields:\n"
            "- product_interest_stated: true if the user has clearly stated interest in a product type (e.g., phone, laptop), false otherwise.\n"
            "- brand_preference_stated: true if the user has mentioned a specific brand preference (e.g., Samsung, Dell), false otherwise.\n"
            "- provided_contact_info: true if the user has provided any contact information (such as email or phone number), false otherwise.\n"
            "- questions_stated: true if the user has asked any product-related questions (especially from the provided questions list), false otherwise.\n"
            "Set each to true if the user's message indicates that state, otherwise false. "
            "Only respond with the JSON object."
        )
        # Compose user prompt with context
        user_prompt = (
            f"Persona: {self.persona_type}, Product: {self.product_type}, Brand: {self.preferred_brand}, "
            f"Questions: {self.questions}\n"
            f"User message: {response}\n"
            "Output JSON:"
        )
        # Call OpenAI API
        api_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            response_format={"type": "json_object"},
            temperature=0,
            max_tokens=100,
        )
        try:
            state = json.loads(api_response.choices[0].message.content)
            self.product_interest_stated = bool(state.get("product_interest_stated", self.product_interest_stated))
            self.brand_preference_stated = bool(state.get("brand_preference_stated", self.brand_preference_stated))
            self.provided_contact_info = bool(state.get("provided_contact_info", self.provided_contact_info))
            self.questions_stated = bool(state.get("questions_stated", self.questions_stated))
        except Exception as e:
            # Fallback: do not update state if parsing fails
            print(f"State extraction failed: {e}")


def simulate_conversation(workflow_agent, user_simulator, max_turns=10):
    """
    Simulate a conversation between the workflow agent and user simulator

    Args:
        workflow_agent: Function that takes a user message and returns an agent response
        user_simulator: VietnameseUserSimulator instance
        max_turns: Maximum number of conversation turns

    Returns:
        List of conversation turns
    """
    conversation = []

    # Start conversation with a greeting from the workflow agent
    agent_message = "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?"
    user_message = user_simulator.generate_response(agent_message)
    conversation.append({"role": "user", "content": user_message})

    # Continue conversation for up to max_turns
    for _ in range(max_turns - 1):
        # Get agent response
        agent_message = workflow_agent(user_message)
        conversation.append({"role": "assistant", "content": agent_message})

        # Check if conversation should end
        if (
            "liên hệ" in agent_message.lower()
            and "cảm ơn" in agent_message.lower()
            and user_simulator.provided_contact_info
        ):
            # Agent has thanked user for contact info and promised follow-up
            break

        # Get user response
        user_message = user_simulator.generate_response(agent_message)
        conversation.append({"role": "user", "content": user_message})

    return conversation

# Example usage:
def example_workflow_agent(user_message: str, user, thread) -> str:
    """
    Example workflow agent implementation (placeholder for your actual agent)
    This is just to demonstrate how to use the user simulator
    """
    # This is where you would connect to your actual workflow agent
    # For demonstration purposes, we'll use a simple rule-based response

    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(user_message)}],
        user_id=user.id,
    )


# Run simulation
if __name__ == "__main__":
    # Create different persona types for demonstration
    phone_test_generation_call = wandb_client.create_call(
        op="phone_test_generation_call",inputs={"test_type": "phone_test_generation_call"}
    )
    personas = ["random"]

    for persona in personas:
        user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
        thread = create_thread(CreateThreadModel(id = uuid4(),
            user_id=user.id, name=user.user_name))
        print(f"\n{'='*50}")
        print(f"SIMULATING CONVERSATION WITH {persona.upper()} PERSONA")
        print(f"user_id: {user.id}")
        print(f"thread_id: {thread.id}")
        print(f"{'='*50}\n")

        user_sim = VietnameseUserSimulator(persona_type=persona)
        conversation = simulate_conversation(lambda msg: example_workflow_agent(msg, user, thread), user_sim)

        
        print("\nCONVERSATION:")

        for turn in conversation:
            if turn["role"] == "assistant":
                print(f"\nAgent: {turn['content']}")
            else:
                print(f"\nUser: {turn['content']}")

        print(f"\nContact info provided: {user_sim.provided_contact_info}")
        print(f"Product interest stated: {user_sim.product_interest_stated}")
        print(f"Brand preference stated: {user_sim.brand_preference_stated}")
        print(f"questions stated: {user_sim.questions_stated}")
        
    wandb_client.finish_call(phone_test_generation_call, output=None)


SIMULATING CONVERSATION WITH RANDOM PERSONA
user_id: 56cd81c1-2c69-4897-8cf0-f9676006eb20
thread_id: 8924048a-183d-41a2-996d-8dbc4084aa43

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196e42c-6f31-7022-a3cc-d5904e85c9d2
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196e42c-6f31-7022-a3cc-d5904e85c9d2
2025-05-18 16:14:03 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-18 16:14:03 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Persona data: {
  "name": "Nguyễn Văn Huy",
  "gender": "male",
  "email": "huynguyen123@gmail.com",
  "phone": "0987654321",
  "age": 35,
  "occupation": "Freelance Graphic Designer",
  "product_type": "laptop",
  "budget": 15000000,
  "preferred_brand": "Dell",
  "technical_knowledge": "high",
  "verbosity": "minimal",
  "patience": "high",
  "willingness_to_share_info": "low",
  "questions": [
    "Có card đồ họa rời không?",
    "Thời gian sử dụng pin là bao lâu?",
    "Có hỗ trợ nâng 